# Simple Example

This notebook demonstrates how to use the `data_agents` package with data from two sources.

## 1. Import Required Packages

First, we need to import the necessary packages.

In [ ]:
import data_agents as da

## 2. Authenticate

The `Authenticate()` function is designed to handle authentication for various available services.

In [ ]:
da.Authenticate()

## 3. Create a Feature Collection from a CSV file

The `FeatureCollection` class is used to work with sets of properties related to specific `Geometry`s. Here, we create a `FeatureCollection` from a dictionary read from a CSV file.

In [ ]:
muri_coring_fc = da.FeatureCollection.from_csv(
    "../etc/nasa_power_example/muri_coring_locations.csv",
    da.Geometry.to_point(["Latitude", "Longitude"]),
)

# print all contents of the feature collection
print(muri_coring_fc.get_info())

# print just the properties available in the feature collection
print(list(muri_coring_fc.properties().keys()))

## 4. Create a Feature Collection from an External Service

We can also create a `FeatureCollection` based on an external data source. Here, we get a `FeatureCollection` for NASA POWER data.

In [ ]:
nasa_power_fc = da.FeatureCollection.from_service("NASA_POWER/DAILY/AG")

# print the properties available from the NASA POWER FeatureCollection related to
# temperature and precipitation
print([(key, val["name"]) for key, val in nasa_power_fc.properties(".*temp.*").items()])
print(
    [(key, val["name"]) for key, val in nasa_power_fc.properties(".*precip.*").items()]
)

## 5. Filter and Join the Feature Collections

Filters and joins can be used in combination on one or more `FeatureCollection`. Here, we match records from CSV file with the closest NASA POWER measurement within 5 km. We also restrict measurement data to the year 2024.

In [ ]:
# define a distance-based spatial filter to use with a join to match points within 5 km
spatial_filter = da.Filter.within_distance(
    left_field=".geo",
    right_field=".geo",
    distance=5000,  # distance in meters
)

# define a join that saves the best (closest) match only
save_best_join = da.Join.save_best(
    matches_key="best_nasa_measurement", distance_key="distance"
)

# apply the join to link each feature in the Muri Coring collection to NASA POWER
# features within 5 km and within the year 2024
joined_fc = save_best_join.apply(muri_coring_fc, nasa_power_fc, spatial_filter).filter(
    da.Filter.date(".date", "2024-01-01", "2024-12-31")
)

## 6. Map and Reduce Feature Collections

Maps are used to calculate Feature properties using custom algorithms. Reducers are used to calculate things like means and variance. Here, we map a custom function to calculate potential evapotranspiration for our dataset. We then use Reducers to calculate mean temperature and radiation, along with total annual precipitation.

## 7. Plotting

The `get_info()` function is used to access Feature Collection data.

In [ ]:
# plot the results
print(joined_fc.get_info())

## Summary

This notebook demonstrated how to use the `data_agents` package with data from two sources:

1. **Authentication**: Authenticate the user with connected services